# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import json 
from pprint import pprint

# Import API key
from api_keys import gm_api

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

def  K_to_C(x):
    return x-273.15

city_data_df['Max Temp'] = city_data_df['Max Temp'].apply(K_to_C)

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,mahebourg,-20.4081,57.7000,22.92,83,20,2.06,MU,1667857480
1,dikson,73.5069,80.5464,-15.12,94,100,4.95,RU,1667857440
2,sapa,5.1544,120.3241,26.41,82,66,1.09,PH,1667857744
3,mataura,-46.1927,168.8643,17.40,53,78,1.29,NZ,1667857428
4,ushuaia,-54.8000,-68.3000,16.81,45,75,7.72,AR,1667857439


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
# Configure the map plot
gmaps.configure(api_key=gm_api)
fig= gmaps.figure()
fig_layout= {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


location= city_data_df[["Lat", "Lng"]]
humidity= city_data_df["Humidity"].astype(float)
humid_point= gmaps.heatmap_layer(location, weights=humidity)
fig.add_layer(humid_point)
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather= city_data_df[(city_data_df['Max Temp']<27)][(city_data_df['Max Temp']>21)][(city_data_df["Wind Speed"]<4.5)][(city_data_df["Cloudiness"]==0)]



# Drop any rows with null values
ideal_weather= ideal_weather.dropna()

# Display sample data
ideal_weather

C:\Users\Adria\AppData\Local\Temp\ipykernel_11852\468030053.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_weather= city_data_df[(city_data_df['Max Temp']<27)][(city_data_df['Max Temp']>21)][(city_data_df["Wind Speed"]<4.5)][(city_data_df["Cloudiness"]==0)]


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
12,bethel,41.3712,-73.4140,23.20,17,0,4.12,US,1667857263
164,mount isa,-20.7333,139.5000,26.87,44,0,0.00,AU,1667857784
187,buraydah,26.3260,43.9750,21.25,66,0,3.31,SA,1667857576
227,adrar,20.5022,-10.0711,25.84,15,0,3.91,MR,1667857800
291,umm lajj,25.0213,37.2685,23.75,60,0,3.06,SA,1667857630
336,grand gaube,-20.0064,57.6608,22.99,71,0,1.34,MU,1667857636
393,mormugao,15.4000,73.8000,24.12,61,0,2.47,IN,1667857851
427,mayo,38.8876,-76.5119,25.23,42,0,3.60,US,1667857423
477,cascavel,-24.9558,-53.4553,22.78,25,0,1.03,BR,1667857877


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""



# Display sample data
hotel_df

C:\Users\Adria\AppData\Local\Temp\ipykernel_11852\1243747654.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
12,bethel,US,41.3712,-73.4140,17,
164,mount isa,AU,-20.7333,139.5000,44,
187,buraydah,SA,26.3260,43.9750,66,
227,adrar,MR,20.5022,-10.0711,15,
291,umm lajj,SA,25.0213,37.2685,60,
336,grand gaube,MU,-20.0064,57.6608,71,
393,mormugao,IN,15.4000,73.8000,61,
427,mayo,US,38.8876,-76.5119,42,
477,cascavel,BR,-24.9558,-53.4553,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "keyword": "hotel",
    "types": "hotel",
    "radius": 10000,
    "key": gm_api
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat= row["Lat"]
    lng= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url =  "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hilton Garden Inn Danbury
mount isa - nearest hotel: Redearth Boutique Hotel
buraydah - nearest hotel: Rollins Hotel
adrar - nearest hotel: No hotel found
umm lajj - nearest hotel: Waves Hotel
grand gaube - nearest hotel: Royal Palm Beachcomber Luxury
mormugao - nearest hotel: Hotel Z Square
mayo - nearest hotel: Historic Inns of Annapolis
cascavel - nearest hotel: Hotel Caiuá
bilma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
12,bethel,US,41.3712,-73.4140,17,Hilton Garden Inn Danbury
164,mount isa,AU,-20.7333,139.5000,44,Redearth Boutique Hotel
187,buraydah,SA,26.3260,43.9750,66,Rollins Hotel
227,adrar,MR,20.5022,-10.0711,15,No hotel found
291,umm lajj,SA,25.0213,37.2685,60,Waves Hotel
336,grand gaube,MU,-20.0064,57.6608,71,Royal Palm Beachcomber Luxury
393,mormugao,IN,15.4000,73.8000,61,Hotel Z Square
427,mayo,US,38.8876,-76.5119,42,Historic Inns of Annapolis
477,cascavel,BR,-24.9558,-53.4553,25,Hotel Caiuá


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
gmaps.configure(api_key=gm_api)
fig= gmaps.figure()
fig_layout= {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

hotel_loc= hotel_df[["Lat", "Lng"]]
locations= hotel_df["Hotel Name"].tolist()
marker= gmaps.marker_layer(hotel_loc, info_box_content=locations)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))